In [1]:
%cd ..

/Users/xiruihan/Desktop/RecFldGrn-main


# Load Data


## Load Raw

In [5]:
import os 
import pandas as pd

RecName = 'Smoking'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/Smoking.csv


In [6]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,DT,V
0,P0,EC23,15471,2023-03-19,B
1,P0,EC142,21622,2023-03-20,D
2,P0,EC113,19101,2023-03-23,A
3,P0,EC166,22867,2023-03-26,B
4,P0,EC91,19104,2023-03-27,D
...,...,...,...,...,...
96,P5,EC129,20285,2023-04-16,B
97,P6,EC92,14290,2023-03-19,D
98,P6,EC123,17942,2023-03-21,A
99,P6,EC15,17943,2023-03-28,B


## Select Columns 

In [8]:
parent_ids = ['PID','ECID']
focal_ids = ['RID']


selected_cols = ['V', 'DT']

df = df[parent_ids + focal_ids + selected_cols]
df

,PID,ECID,RID,V,DT
0,P0,EC23,15471,B,2023-03-19
1,P0,EC142,21622,D,2023-03-20
2,P0,EC113,19101,A,2023-03-23
3,P0,EC166,22867,B,2023-03-26
4,P0,EC91,19104,D,2023-03-27
...,...,...,...,...,...
96,P5,EC129,20285,B,2023-04-16
97,P6,EC92,14290,D,2023-03-19
98,P6,EC123,17942,A,2023-03-21
99,P6,EC15,17943,B,2023-03-28


## Basic Filtering 

In [ ]:
# nothing

## DF Rec Raw

In [22]:
df_RecRaw = df
df_RecRaw

,PID,ECID,RID,V,DT
0,P4,EC10,16453.0,D,2023-04-15
1,P3,EC101,20384.0,A,2023-04-16
2,P3,EC102,20385.0,D,2023-03-28
3,P5,EC105,Missing1.0,NaN,NaN
4,P4,EC106,Missing2.0,NaN,NaN
...,...,...,...,...,...
150,P6,EC93,15484.0,A,2023-04-04
151,P5,EC94,Missing90.0,NaN,NaN
152,P2,EC95,Missing91.0,NaN,NaN
153,P3,EC98,15447.0,A,2023-04-13


# Fields in Records

## Field: V

In [23]:
df['V']

0        D
1        A
2        D
3      NaN
4      NaN
      ... 
150      A
151    NaN
152    NaN
153      A
154    NaN
Name: V, Length: 155, dtype: object

## Field: DT

In [24]:
df['DT']

0      2023-04-15
1      2023-04-16
2      2023-03-28
3             NaN
4             NaN
          ...    
150    2023-04-04
151           NaN
152           NaN
153    2023-04-13
154           NaN
Name: DT, Length: 155, dtype: object

## Call Parents

In [25]:
def load_df_data_from_folder(data_folder, IDName):
    file_list = os.listdir(data_folder)
    # file_lis
    df = pd.concat([pd.read_pickle(os.path.join(data_folder, i)) for i in file_list if '.p' in i]).reset_index(drop = True)
    df = df.sort_values(IDName).reset_index(drop = True)
    return df# .shape

fullrec_folder = 'data/ProcData/RecFolder/EC'
df_Pat = load_df_data_from_folder(fullrec_folder, 'ECID')[['ECID','PID']]
print(df_Pat.shape)
df_Pat.head()
print(df_Pat)


df = pd.merge(df_Pat, df_RecRaw, on = parent_ids, how = 'left')
df

(151, 2)
           ECID PID
0          EC10  P4
1         EC101  P3
2         EC102  P3
3         EC105  P5
4         EC106  P4
..          ...  ..
146        EC93  P6
147        EC94  P5
148        EC95  P2
149        EC98  P3
150  Missing1.0  P7

[151 rows x 2 columns]


,ECID,PID,RID,V,DT
0,EC10,P4,16453.0,D,2023-04-15
1,EC101,P3,20384.0,A,2023-04-16
2,EC102,P3,20385.0,D,2023-03-28
3,EC105,P5,Missing1.0,NaN,NaN
4,EC106,P4,Missing2.0,NaN,NaN
...,...,...,...,...,...
150,EC93,P6,15484.0,A,2023-04-04
151,EC94,P5,Missing90.0,NaN,NaN
152,EC95,P2,Missing91.0,NaN,NaN
153,EC98,P3,15447.0,A,2023-04-13


In [26]:
import numpy as np

d = df
for col in focal_ids: 
    d.loc[d[col].isnull(), col] = ['Missing' + str(i) for i in np.ones(d[col].isnull().sum()).cumsum()]
    # d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,ECID,PID,RID,V,DT
0,EC10,P4,16453.0,D,2023-04-15
1,EC101,P3,20384.0,A,2023-04-16
2,EC102,P3,20385.0,D,2023-03-28
3,EC105,P5,Missing1.0,NaN,NaN
4,EC106,P4,Missing2.0,NaN,NaN
...,...,...,...,...,...
150,EC93,P6,15484.0,A,2023-04-04
151,EC94,P5,Missing90.0,NaN,NaN
152,EC95,P2,Missing91.0,NaN,NaN
153,EC98,P3,15447.0,A,2023-04-13


## Pin Down

In [27]:
df_proc = df[['PID', 'ECID', 'RID', 'V','DT']]
df_proc

,PID,ECID,RID,V,DT
0,P4,EC10,16453.0,D,2023-04-15
1,P3,EC101,20384.0,A,2023-04-16
2,P3,EC102,20385.0,D,2023-03-28
3,P5,EC105,Missing1.0,NaN,NaN
4,P4,EC106,Missing2.0,NaN,NaN
...,...,...,...,...,...
150,P6,EC93,15484.0,A,2023-04-04
151,P5,EC94,Missing90.0,NaN,NaN
152,P2,EC95,Missing91.0,NaN,NaN
153,P3,EC98,15447.0,A,2023-04-13


# Save and Load

## Save

In [28]:
def convert_PID_to_PIDgroup(x, RANGE_SIZE):
    if type(x) == str:
        if 'P' in x:
            int_value = int(x.replace('P', ''))
    else:
        int_value = int(x)
    range_size = RANGE_SIZE
    group_idx = int(int_value / range_size)
    s = group_idx * range_size
    e = (group_idx + 1) * range_size - 1
    groupname = f'Group_s{s}_e{e}'
    return groupname

def write_df_to_folders(DataName, data_folder, df, IDname):
    dfy = df.copy()
    # data_folder = f'../Data/B-EHRDataSet/{DataName}'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    dfy['GroupName'] = dfy[IDname].apply(lambda x: convert_PID_to_PIDgroup(x , RANGE_SIZE))
    for groupname, dfx in dfy.groupby('GroupName'):
        file = os.path.join(data_folder, f'{groupname}.p')
        # print(file)
        dfx = dfx.drop(columns= ['GroupName'])
        dfx.to_pickle(file)
        
RANGE_SIZE = 10000
        

In [29]:
# the folder to save the Record.
###############
RecName = 'Smoking'
###############





rec_folder = 'data/ProcData/RecFolder'
IDname = 'PID' # always be PID, not review ID or other ID.

fullrec_folder = os.path.join(rec_folder, RecName)
print(fullrec_folder, '<----- fullrec_folder to save the records')
if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)

data/ProcData/RecFolder/Smoking <----- fullrec_folder to save the records


In [30]:
df = df_proc

write_df_to_folders(RecName, fullrec_folder, df, IDname)

## Load

In [31]:
def load_df_data_from_folder(data_folder, IDName):
    file_list = os.listdir(data_folder)
    # file_lis
    df = pd.concat([pd.read_pickle(os.path.join(data_folder, i)) for i in file_list if '.p' in i]).reset_index(drop = True)
    # df = df.sort_values(IDName).reset_index(drop = True)
    return df# .shape

In [32]:
IDName = 'PID'
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder, IDName)
print(df.shape)
df.head()
df

data/ProcData/RecFolder/Smoking
(155, 5)


,PID,ECID,RID,V,DT
0,P4,EC10,16453.0,D,2023-04-15
1,P3,EC101,20384.0,A,2023-04-16
2,P3,EC102,20385.0,D,2023-03-28
3,P5,EC105,Missing1.0,NaN,NaN
4,P4,EC106,Missing2.0,NaN,NaN
...,...,...,...,...,...
150,P6,EC93,15484.0,A,2023-04-04
151,P5,EC94,Missing90.0,NaN,NaN
152,P2,EC95,Missing91.0,NaN,NaN
153,P3,EC98,15447.0,A,2023-04-13


## Load One Sample

In [33]:
def get_dfx_p_filename(IDValue, DataName, data_folder, RANGE_SIZE):
    group_name = convert_PID_to_PIDgroup(IDValue, RANGE_SIZE) 
    p_filename = f'{data_folder}/{DataName}/{group_name}.p'
    return p_filename
    
def load_patient_data(RID, IDValue, dfx):
    dfx = dfx[dfx[RID] == IDValue].reset_index(drop = True)
    return dfx

def get_dfx_from_buffer(RID, IDValue, DataName, data_folder, RANGE_SIZE, BUCKET_buffer):
    p_filename = get_dfx_p_filename(IDValue, DataName, data_folder, RANGE_SIZE)
    if p_filename not in BUCKET_buffer:
        dfx = pd.read_pickle(p_filename)
        BUCKET_buffer[p_filename] = dfx
        if len(BUCKET_buffer) > 100:
            for key in random.sample(BUCKET_buffer.keys(), 4):
                del BUCKET_buffer[key]
    else:
        dfx = BUCKET_buffer[p_filename]
    return dfx, BUCKET_buffer


class DataPoint(object):

    '''
    DataPoint: stands for one data point. 
    one instance is a patient.
    the instance of the patient could quickly the his/her record information by using `get_df_rec`.
    '''
    BUCKET_buffer = {}

    def __init__(self, RID, IDValue, data_folder, RANGE_SIZE):
        self.RID = RID
        self.IDValue = IDValue
        self.data_folder = data_folder
        self.df_rec_dict = {}
        self.RANGE_SIZE = RANGE_SIZE
        
    def get_df_rec(self, DataName):
        '''
        DataName: the name of the record, like: P, EC, BMI, PN, PNSect, etc.
        '''
        if DataName not in self.df_rec_dict:
            dfx, BUCKET_buffer = get_dfx_from_buffer(self.RID, self.IDValue, 
                                                     DataName, 
                                                     self.data_folder, 
                                                     self.RANGE_SIZE, 
                                                     self.BUCKET_buffer)
            self.BUCKET_buffer = BUCKET_buffer
            df_p = load_patient_data(self.RID, self.IDValue, dfx)
            self.df_rec_dict[DataName] = df_p
            return df_p
        else:
            return self.df_rec_dict[DataName]
        
    def __repr__(self):
        return f'<{self.RID} [{self.IDValue}]>'